In [27]:
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib.pyplot as plt
import seaborn as sns
# Enable inline plotting
%matplotlib inline

In [28]:
train_data = pd.read_csv('./data/train_values.csv')
train_label= pd.read_csv('./data/train_labels.csv')
test_data= pd.read_csv('./data/test_values.csv')

In [29]:
train_data.columns

Index(['patient_id', 'slope_of_peak_exercise_st_segment', 'thal',
       'resting_blood_pressure', 'chest_pain_type', 'num_major_vessels',
       'fasting_blood_sugar_gt_120_mg_per_dl', 'resting_ekg_results',
       'serum_cholesterol_mg_per_dl', 'oldpeak_eq_st_depression', 'sex', 'age',
       'max_heart_rate_achieved', 'exercise_induced_angina'],
      dtype='object')

In [30]:
train_data[:10]

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0
5,ldukkw,1,normal,130,3,0,0,0,180,0.0,1,42,150,0
6,2gbyh9,2,reversible_defect,150,4,2,0,2,258,2.6,0,60,157,0
7,daa9kp,2,fixed_defect,150,4,1,0,2,276,0.6,1,57,112,1
8,3nwy2n,3,reversible_defect,170,4,0,0,2,326,3.4,1,59,140,1
9,1r508r,2,normal,120,3,0,0,0,219,1.6,0,50,158,0


In [31]:
feature_names = ['slope_of_peak_exercise_st_segment', 'resting_blood_pressure', 'chest_pain_type', 'num_major_vessels', 'fasting_blood_sugar_gt_120_mg_per_dl', 'resting_ekg_results', 'serum_cholesterol_mg_per_dl', 'sex','max_heart_rate_achieved','exercise_induced_angina']


In [32]:
X = train_data[feature_names]


In [33]:
Y= train_label['heart_disease_present']

In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

In [35]:
scaler = preprocessing.MinMaxScaler()
X_scaler = scaler.fit_transform(X)
X_scaler.shape

(180, 10)

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaler, Y,  random_state=0)

In [37]:
model = SVC(C=1, kernel='rbf', degree=1,coef0=3.0, probability=True)
model2= KNeighborsClassifier(n_neighbors=1, weights='uniform')


In [38]:
import numpy as np
from sklearn.metrics import log_loss
names = []
scores = []
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
scores.append(accuracy_score(Y_test, y_pred))
scores.append(log_loss(Y_test,y_pred,eps=1e-15))
tr_split = pd.DataFrame({'Score': scores})
print(tr_split)

      Score
0  0.822222
1  6.140334


In [50]:
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Conv2D,MaxPooling2D, Flatten
from keras.layers.advanced_activations import ELU
#Single layer architecture
model = Sequential()
model.add(Dense(32,input_shape=(X_train.shape[1],)))
    # fully connected 2 with dropout
model.add(Dropout(0.2))
model.add(ELU())
model.add(Dense(64,input_shape=(X_train.shape[1],)))
model.add(ELU())
model.add(Dense(64,input_shape=(X_train.shape[1],)))
model.add(ELU())
model.add(Dense(128,input_shape=(X_train.shape[1],)))
model.add(ELU())
model.add(Dense(128,input_shape=(X_train.shape[1],)))
model.add(ELU())
model.add(Dense(256,input_shape=(X_train.shape[1],)))
model.add(ELU())
model.add(Dropout(0.5))
model.add(Dense(256,input_shape=(X_train.shape[1],)))
model.add(ELU())
model.add(Dropout(0.7))
model.add(Dense(2))
model.add(Activation('softmax'))
print(model.summary())
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=128, epochs=100, verbose=1, validation_data=(X_test, Y_test))
loss, accuracy = model.evaluate(X_test,Y_test, verbose=0)
print("Loss : "+str(loss))
print("Accuracy :"+str(accuracy*100.0))
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,Y_train)
print(clf.score(X_test,Y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_118 (Dense)            (None, 32)                352       
_________________________________________________________________
dropout_39 (Dropout)         (None, 32)                0         
_________________________________________________________________
elu_90 (ELU)                 (None, 32)                0         
_________________________________________________________________
dense_119 (Dense)            (None, 64)                2112      
_________________________________________________________________
elu_91 (ELU)                 (None, 64)                0         
_________________________________________________________________
dense_120 (Dense)            (None, 64)                4160      
_________________________________________________________________
elu_92 (ELU)                 (None, 64)                0         
__________

135/135 [==============================] - 0s 148us/step - loss: 0.4290 - acc: 0.8222 - val_loss: 0.5277 - val_acc: 0.7333
Epoch 40/100
135/135 [==============================] - 0s 227us/step - loss: 0.5007 - acc: 0.7481 - val_loss: 0.5792 - val_acc: 0.7111
Epoch 41/100
135/135 [==============================] - 0s 314us/step - loss: 0.4390 - acc: 0.8148 - val_loss: 0.6362 - val_acc: 0.6889
Epoch 42/100
135/135 [==============================] - 0s 170us/step - loss: 0.4904 - acc: 0.7852 - val_loss: 0.6329 - val_acc: 0.6889
Epoch 43/100
135/135 [==============================] - 0s 113us/step - loss: 0.4539 - acc: 0.8296 - val_loss: 0.5666 - val_acc: 0.7111
Epoch 44/100
135/135 [==============================] - 0s 132us/step - loss: 0.4508 - acc: 0.8074 - val_loss: 0.5053 - val_acc: 0.7778
Epoch 45/100
135/135 [==============================] - 0s 125us/step - loss: 0.4105 - acc: 0.7778 - val_loss: 0.4680 - val_acc: 0.8000
Epoch 46/100
135/135 [==============================] - 0s 19

Epoch 99/100
135/135 [==============================] - 0s 289us/step - loss: 0.4669 - acc: 0.7704 - val_loss: 0.6417 - val_acc: 0.7111
Epoch 100/100
135/135 [==============================] - 0s 235us/step - loss: 0.4355 - acc: 0.8074 - val_loss: 0.6758 - val_acc: 0.7111
Loss : 0.6758437249395582
Accuracy :71.1111119058397
0.6
